In [1]:
# === INSTALLS ===
!pip -q install transformers torch pandas gradio==4.44.1

# === IMPORTS ===
import re, json, math, datetime as dt
import pandas as pd
from dataclasses import dataclass, asdict
from typing import List, Dict, Any
from transformers import pipeline


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.2/131.2 kB 11.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.41.0 requires websockets<15.1.0,>=13.0.0, but you have websockets 12.0 which is incompatible.
dataproc-spark-connect 0.8.3 requires websockets>=14.0, but you have websockets 12.0 which is incompatible.
google-adk 1.14.1 requires websockets<16.0.0,>=15.0.1, but you have websockets 12.0 which is incompatible.
yfinance 0.2.66 requires websockets>=13.0, but you have websockets 12.0 which is incompatible.


In [2]:
!pip install "websockets<13.0,>=10.0"

In [3]:
# === SAMPLE UNSTRUCTURED REPORTS ===
RADIOL_REPORT_1 = """
MRI Brain (2025-09-10): An enhancing lesion measuring 2.8 x 2.3 cm in the right temporal lobe.
Compared to prior scan, there is progressive increase in size. No midline shift.
No cervical lymphadenopathy noted. Impression: likely high-grade neoplasm.
"""

PATH_REPORT_1 = """
Surgical pathology (2025-09-12): High-grade glioma. Ki-67 proliferation index: 35%.
IDH1 mutation: negative (wildtype). TP53 mutation detected. EGFR L858R detected with VAF 25%.
Comment: Rapid growth features present.
"""

# === SAMPLE STRUCTURED DATA ===
labs_df = pd.DataFrame({
    "Test": ["CEA","LDH","WBC","Hemoglobin"],
    "Value":[8.9, 220, 7.2, 12.6],
    "Unit" :["ng/mL","U/L","x10^9/L","g/dL"],
    "Date" :["2025-09-11","2025-09-11","2025-09-11","2025-09-11"]
})
labs_df.to_csv("labs.csv", index=False)

genomics_df = pd.DataFrame({
    "Gene":["TP53","IDH1","EGFR"],
    "Variant":["p.R273H","WildType","L858R"],
    "VAF":[0.32,0.0,0.25]
})
genomics_df.to_csv("genomics.csv", index=False)

print("Sample data files written: labs.csv, genomics.csv")


Sample data files written: labs.csv, genomics.csv


In [4]:
# === PRETRAINED BIOMED NER (no fine-tuning; fast) ===
ner = pipeline("ner", model="d4data/biomedical-ner-all", aggregation_strategy="simple")

size_pat = re.compile(r'(\d+(?:\.\d+)?)\s*(?:x\s*(\d+(?:\.\d+)?))?\s*cm', re.I)
ki67_pat = re.compile(r'ki[-\s]?67[^0-9%]*?(\d{1,3})\s*%', re.I)
ln_pat   = re.compile(r'\b(lymph\s*node|lymphadenopathy)\b.*?\b(no|not|absent|negative)\b', re.I)
growth_pat = re.compile(r'\b(progressive|increasing|rapid|slow|stable)\b', re.I)
site_pat = re.compile(r'(frontal|parietal|temporal|occipital|cerebellar|brainstem|right|left)\s+(?:lobe|hemisphere)?', re.I)
mut_pat = re.compile(r'\b(EGFR|IDH1|TP53)\b[^.,;:()]*?(L858R|R132H|WildType|WT|p\.[A-Z]\d+[A-Z])?', re.I)

def extract_text_entities(radiology_text: str, pathology_text: str) -> Dict[str, Any]:
    # NER (not strictly needed for our minimal fields, but useful to show capability)
    _ = ner(radiology_text + "\n" + pathology_text)  # result not heavily used in MVP

    # Size (take max dimension)
    sizes = size_pat.findall(radiology_text)
    max_dim = None
    if sizes:
        dims = []
        for a,b in sizes:
            a = float(a)
            if b: dims.append(max(a, float(b)))
            else: dims.append(a)
        max_dim = max(dims) if dims else None

    # Site
    site_match = site_pat.search(radiology_text)
    site = site_match.group(0).strip().lower() if site_match else None
    site = site.replace(" hemisphere","").replace(" lobe","") if site else None

    # Growth descriptors
    growth = None
    for m in growth_pat.findall(radiology_text + " " + pathology_text):
        growth = m.lower()  # keep the last meaningful occurrence

    # Lymph nodes (simple negation detection)
    ln_involved = None
    if ln_pat.search(radiology_text):
        ln_involved = False
    elif re.search(r'\b(lymphadenopathy|nodes?)\b.*\b(present|enlarged|positive)\b', radiology_text, re.I):
        ln_involved = True

    # Pathology: histology & Ki-67
    histology = "high-grade glioma" if re.search(r'high[-\s]?grade.*glioma', pathology_text, re.I) else None
    ki = ki67_pat.search(pathology_text)
    ki67 = int(ki.group(1)) if ki else None

    # Mutations
    muts = {}
    for g,m in mut_pat.findall(pathology_text):
        g = g.upper()
        m = m.upper() if m else None
        # normalize
        if g=="IDH1" and ("WILDTYPE" in (m or "") or m=="WT"): m = "WILDTYPE"
        if g not in muts: muts[g] = set()
        if m: muts[g].add(m)
    muts = {g: sorted(list(v)) for g,v in muts.items()}

    return {
        "tumor_size_cm": max_dim,
        "site": site,              # e.g., 'right temporal'
        "growth": growth,          # progressive / stable / slow / rapid
        "lymph_nodes_involved": ln_involved,  # True/False/None
        "histology": histology,
        "Ki67_percent": ki67,
        "mutations": muts
    }

# Quick test on our sample notes
text_entities = extract_text_entities(RADIOL_REPORT_1, PATH_REPORT_1)
text_entities


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/266M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/373 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cuda:0


{'tumor_size_cm': 2.8,
 'site': 'right',
 'growth': 'rapid',
 'lymph_nodes_involved': None,
 'histology': 'high-grade glioma',
 'Ki67_percent': 35,
 'mutations': {'IDH1': [], 'TP53': [], 'EGFR': []}}

In [5]:
def load_structured():
    labs = pd.read_csv("labs.csv")
    genomics = pd.read_csv("genomics.csv")
    labs_dict = {row.Test: {"value": float(row.Value), "unit": row.Unit, "date": row.Date}
                 for _, row in labs.iterrows()}
    genomics_list = genomics.to_dict("records")
    return labs_dict, genomics_list

def build_patient_profile(radiology_text, pathology_text):
    ents = extract_text_entities(radiology_text, pathology_text)
    labs, genomics = load_structured()
    profile = {
        "patient_id": "DEMO-001",
        "demographics": {"age": 58, "sex": "F"},
        "findings": {
            "tumor": {
                "site": ents["site"],
                "max_diameter_cm": ents["tumor_size_cm"],
                "growth": ents["growth"]
            },
            "pathology": {
                "histology": ents["histology"],
                "Ki67_percent": ents["Ki67_percent"],
                "lymph_nodes_involved": ents["lymph_nodes_involved"]
            },
            "genomics": genomics,
            "mutations_extracted_from_text": ents["mutations"],
            "labs": labs
        },
        "provenance": {
            "radiology_report_excerpt": radiology_text.strip()[:350],
            "pathology_report_excerpt": pathology_text.strip()[:350]
        }
    }
    return profile

profile = build_patient_profile(RADIOL_REPORT_1, PATH_REPORT_1)
print(json.dumps(profile, indent=2))


{
  "patient_id": "DEMO-001",
  "demographics": {
    "age": 58,
    "sex": "F"
  },
  "findings": {
    "tumor": {
      "site": "right",
      "max_diameter_cm": 2.8,
      "growth": "rapid"
    },
    "pathology": {
      "histology": "high-grade glioma",
      "Ki67_percent": 35,
      "lymph_nodes_involved": null
    },
    "genomics": [
      {
        "Gene": "TP53",
        "Variant": "p.R273H",
        "VAF": 0.32
      },
      {
        "Gene": "IDH1",
        "Variant": "WildType",
        "VAF": 0.0
      },
      {
        "Gene": "EGFR",
        "Variant": "L858R",
        "VAF": 0.25
      }
    ],
    "mutations_extracted_from_text": {
      "IDH1": [],
      "TP53": [],
      "EGFR": []
    },
    "labs": {
      "CEA": {
        "value": 8.9,
        "unit": "ng/mL",
        "date": "2025-09-11"
      },
      "LDH": {
        "value": 220.0,
        "unit": "U/L",
        "date": "2025-09-11"
      },
      "WBC": {
        "value": 7.2,
        "unit": "x10^9/L",
 

In [6]:
# === STEP 5: DIAGNOSTIC REASONING (fixed) ===

from dataclasses import dataclass
from typing import List, Dict, Any

@dataclass
class DiagnosisScore:
    name: str
    score: float
    evidence: List[str]

def diagnose(profile: Dict[str, Any]) -> List[DiagnosisScore]:
    """
    Simple transparent reasoning engine.
    Rules are illustrative, not clinical guidelines.
    """
    findings = profile["findings"]
    tumor = findings.get("tumor", {})
    path = findings.get("pathology", {})
    labs = findings.get("labs", {})
    muts_text = findings.get("mutations_extracted_from_text", {})
    genomics = findings.get("genomics", [])

    scores: Dict[str, DiagnosisScore] = {}

    def add(diag, pts, ev):
        """helper to add score + evidence"""
        if diag not in scores:
            scores[diag] = DiagnosisScore(diag, 0.0, [])
        scores[diag].score += pts
        scores[diag].evidence.append(ev)

    # ---- Rules ----
    # 1. Histology indicates high-grade glioma
    if path.get("histology") and "high-grade glioma" in path["histology"].lower():
        add("High-Grade Glioma", 3.0, "Pathology indicates high-grade glioma")

    # 2. Tumor ≥ 2 cm ⇒ neoplasm suspicion
    if tumor.get("max_diameter_cm") and tumor["max_diameter_cm"] >= 2.0:
        add("Intracranial Neoplasm", 1.5,
            f"Lesion size {tumor['max_diameter_cm']} cm")

    # 3. Progressive / rapid growth ⇒ aggressive behavior
    if tumor.get("growth") in {"progressive", "rapid", "increasing"}:
        add("Aggressive Course Likely", 1.5,
            f"Growth described as {tumor['growth']}")

    # 4. Genomic indicators
    for g_entry in genomics:
        gene, variant = g_entry.get("Gene",""), str(g_entry.get("Variant","")).lower()
        if gene == "IDH1" and variant.startswith("wild"):
            add("High-Grade Glioma", 1.0, "IDH1 wildtype (worse prognosis)")
    if "EGFR" in muts_text and "L858R" in muts_text["EGFR"]:
        add("EGFR-Altered Pathway", 1.0, "EGFR L858R detected (text)")

    # 5. Labs
    cea_val = labs.get("CEA", {}).get("value")
    if cea_val and cea_val > 5:
        add("Consider Metastatic Workup", 0.5, f"CEA elevated ({cea_val})")

    # ---- Rank + return ----
    ranked = sorted(scores.values(), key=lambda x: x.score, reverse=True)
    return ranked

# --- Run on current patient profile ---
diagnoses = diagnose(profile)

print("Diagnoses & Evidence:")
for d in diagnoses:
    print(f"  • {d.name} — Score {d.score} — Evidence: {', '.join(d.evidence)}")


Diagnoses & Evidence:
  • High-Grade Glioma — Score 4.0 — Evidence: Pathology indicates high-grade glioma, IDH1 wildtype (worse prognosis)
  • Intracranial Neoplasm — Score 1.5 — Evidence: Lesion size 2.8 cm
  • Aggressive Course Likely — Score 1.5 — Evidence: Growth described as rapid
  • Consider Metastatic Workup — Score 0.5 — Evidence: CEA elevated (8.9)


In [7]:
def detect_anomalies(profile: Dict[str,Any]) -> List[str]:
    a = []
    f = profile["findings"]
    tumor, path, labs = f["tumor"], f["pathology"], f["labs"]

    # Example contradictions
    if tumor.get("growth") in {"slow","stable"} and (path.get("Ki67_percent") or 0) >= 30:
        a.append("Text says slow/stable but Ki-67 is high (≥30%)")

    if path.get("histology") and "low-grade" in path["histology"] and (path.get("Ki67_percent") or 0) >= 20:
        a.append("Low-grade histology vs high Ki-67")

    # Marker vs size (toy heuristic)
    if labs.get("CEA",{}).get("value") is not None and tumor.get("max_diameter_cm") is not None:
        if labs["CEA"]["value"] < 3 and tumor["max_diameter_cm"] >= 3.5:
            a.append("Large lesion but CEA not elevated (check marker relevance)")

    # Lymph node mention
    if path.get("lymph_nodes_involved") is False and re.search(r'rapid|aggressive', profile["provenance"]["pathology_report_excerpt"], re.I):
        a.append("Aggressive wording but lymph nodes reported negative")

    return a

anoms = detect_anomalies(profile)
anoms


[]

In [8]:
def run_assistant(radiology_text, pathology_text, cea, ldh, wbc):
    try:
        # 1. Build patient profile
        profile = build_patient_profile(radiology_text, pathology_text, cea, ldh, wbc)

        # 2. Diagnose
        dx = diagnose(profile)

        # 3. Detect anomalies
        anomalies = detect_anomalies(profile)

        # 4. Return 3 outputs (for the 3 UI boxes)
        return json.dumps(profile, indent=2), dx, anomalies

    except Exception as e:
        import traceback
        print(traceback.format_exc())
        return f"❌ Error: {e}", "", ""


In [10]:
import gradio as gr

def run_assistant(rad_text, path_text, cea, ldh, wbc, hb):
    # write temp CSVs from UI values (simulating structured inputs)
    labs_ui = pd.DataFrame({
        "Test":["CEA","LDH","WBC","Hemoglobin"],
        "Value":[cea, ldh, wbc, hb],
        "Unit":["ng/mL","U/L","x10^9/L","g/dL"],
        "Date":[str(dt.date.today())]*4
    })
    labs_ui.to_csv("labs.csv", index=False)

    # build profile
    prof = build_patient_profile(rad_text, path_text)
    dx = diagnose(prof)
    an = detect_anomalies(prof)

    dx_table = pd.DataFrame([{"Diagnosis": d.name, "Score": round(d.score,2), "Evidence": "; ".join(d.evidence)} for d in dx])
    return json.dumps(prof, indent=2), dx_table, "\n".join(f"• {x}" for x in an) if an else "No anomalies flagged."

demo = gr.Interface(
    fn=run_assistant,
    inputs=[
        gr.Textbox(label="Radiology Report", value=RADIOL_REPORT_1, lines=7),
        gr.Textbox(label="Pathology Report", value=PATH_REPORT_1, lines=7),
        gr.Slider(0, 50, value=8.9, step=0.1, label="CEA (ng/mL)"),
        gr.Slider(50, 1000, value=220, step=1, label="LDH (U/L)"),
        gr.Slider(2, 20, value=7.2, step=0.1, label="WBC (x10^9/L)"),
        gr.Slider(6, 20, value=12.6, step=0.1, label="Hemoglobin (g/dL)")
    ],
    outputs=[
        gr.Textbox(label="Unified Patient Profile (JSON)"),
        gr.Dataframe(label="Ranked Diagnoses + Evidence"),
        gr.Textbox(label="Cross-Modal Anomalies")
    ],
    title="🧬 Multi-Modal Medical Data Fusion Diagnostic Assistant",
    description="Paste reports, tweak labs, and get a ranked differential with anomalies."
)
demo.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://5a238104d2308901bb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [11]:
def prognosis_score(profile: Dict[str,Any]) -> Dict[str, Any]:
    f = profile["findings"]
    score = 0
    reasons = []

    size = f["tumor"].get("max_diameter_cm") or 0
    if size >= 3: score += 2; reasons.append(f"Size {size} cm (≥3)")

    if f["tumor"].get("growth") in {"progressive","rapid"}:
        score += 2; reasons.append("Progressive growth")

    if (f["pathology"].get("Ki67_percent") or 0) >= 30:
        score += 2; reasons.append(f"Ki-67 {f['pathology']['Ki67_percent']}% (≥30)")

    idh = None
    for g in f["genomics"]:
        if g["Gene"]=="IDH1": idh = str(g["Variant"]).lower()
    if idh and idh.startswith("wild"):
        score += 2; reasons.append("IDH1 wildtype")

    # map to bins
    if score >= 6: label = "High risk"
    elif score >= 3: label = "Intermediate risk"
    else: label = "Low risk"

    return {"risk_score": score, "risk_label": label, "reasons": reasons}

prognosis_score(profile)


{'risk_score': 6,
 'risk_label': 'High risk',
 'reasons': ['Progressive growth', 'Ki-67 35% (≥30)', 'IDH1 wildtype']}